In [ ]:
!pip install numpy matplotlib ipywidgets scipy

In [92]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse, Circle, Arc, Rectangle
from ipywidgets import interactive, FloatSlider, Layout

# Constants (all in mm)
MIRROR_LENGTH = 200
DEFAULT_MIRROR_HEIGHT = 1500
MIRROR_ANGLE_DEGREES = -45
PANEL_BASE_HEIGHT = 1300
PANEL_HEIGHT = 300
PANEL_WIDTH = 30

# Constants for the face and semi-oval panel (all in mm)
FACE_WIDTH = 200
FACE_HEIGHT = 150
FACE_Y_OFFSET = 300
EYE_SIZE = 20
PANEL_Y_OFFSET = 5
APERTURE_WIDTH = 120
SEMI_OVAL_WIDTH = FACE_WIDTH + 40
FACE_X_POSITION = 1300

def calculate_mirror_end_position(x_pos: float, angle_degrees: float, length: float) -> tuple:
    """Calculate the end position (x, y) of the mirror."""
    angle_radians = np.radians(angle_degrees)
    end_x = x_pos + length * np.cos(angle_radians)
    end_y = DEFAULT_MIRROR_HEIGHT + length * np.sin(angle_radians)
    return (end_x, end_y)

def calculate_light_path(start_x: float, light_height: float, light_angle: float) -> tuple:
    """Calculate the x and y coordinates of the light path from the source."""
    light_angle_rad = np.radians(light_angle - 90)
    x_coords = np.array([start_x, start_x + 3000 * np.cos(light_angle_rad)])
    y_coords = np.array([light_height, light_height + 3000 * np.sin(light_angle_rad)])
    return x_coords, y_coords

def calculate_intersection(mirror_x: float, light_x: np.ndarray, light_y: np.ndarray, light_angle: float) -> tuple | None:
    """Calculate the intersection of the light ray with the mirror and compute reflection if applicable."""
    mirror_end_x, mirror_end_y = calculate_mirror_end_position(mirror_x, MIRROR_ANGLE_DEGREES, MIRROR_LENGTH)

    dx_mirror = mirror_end_x - mirror_x
    dy_mirror = mirror_end_y - DEFAULT_MIRROR_HEIGHT
    normal_angle = np.arctan2(dy_mirror, dx_mirror) + np.pi / 2

    dx_light = light_x[1] - light_x[0]
    dy_light = light_y[1] - light_y[0]
    a_light, b_light = dy_light, -dx_light
    c_light = a_light * light_x[0] + b_light * light_y[0]

    a_mirror, b_mirror = dy_mirror, -dx_mirror
    c_mirror = a_mirror * mirror_x + b_mirror * DEFAULT_MIRROR_HEIGHT

    determinant = a_mirror * b_light - a_light * b_mirror
    if determinant != 0:
        x_intersect = (b_light * c_mirror - b_mirror * c_light) / determinant
        y_intersect = (a_mirror * c_light - a_light * c_mirror) / determinant

        if min(light_x) <= x_intersect <= max(light_x) and min(mirror_x, mirror_end_x) <= x_intersect <= max(mirror_x, mirror_end_x):
            incidence_angle = np.arctan2(dy_light, dx_light)
            mirror_angle = np.arctan2(dy_mirror, dx_mirror)
            reflection_angle = 2 * mirror_angle - incidence_angle

            reflected_x = np.array([x_intersect, x_intersect + 3000 * np.cos(reflection_angle)])
            reflected_y = np.array([y_intersect, y_intersect + 3000 * np.sin(reflection_angle)])
            return (reflected_x, reflected_y, (x_intersect, y_intersect))

    return None

def plot_panel(ax: plt.Axes, panel_x: float) -> tuple:
    """Plot a vertical panel at the specified x position and return its coordinates."""
    y_start = PANEL_BASE_HEIGHT
    y_end = PANEL_BASE_HEIGHT + PANEL_HEIGHT
    ax.add_patch(plt.Rectangle((panel_x, y_start), PANEL_WIDTH, PANEL_HEIGHT, color='gray', label='Opaque Panel'))
    return (panel_x, y_start, panel_x + PANEL_WIDTH, y_end)

def calculate_panel_intersection(light_x: np.ndarray, light_y: np.ndarray, panel_x: float) -> tuple | None:
    """Calculate the intersection of the light path with a vertical panel."""
    dx_light = light_x[1] - light_x[0]
    if dx_light == 0:  # Avoid division by zero if light is vertical
        return None
    
    slope_light = (light_y[1] - light_y[0]) / dx_light
    intercept_light = light_y[0] - slope_light * light_x[0]

    y_intersect = slope_light * panel_x + intercept_light
    if min(light_x) <= panel_x <= max(light_x) and PANEL_BASE_HEIGHT <= y_intersect <= PANEL_BASE_HEIGHT + PANEL_HEIGHT:
        return (panel_x, y_intersect)
    return None

def plot_face_and_panel(ax: plt.Axes, mirror_y: float, face_offset: float):
    """Plot an oval face with an eye looking towards the mirror and a semi-oval panel above it."""
    face_center_x = FACE_X_POSITION + face_offset * (SEMI_OVAL_WIDTH - FACE_WIDTH) / 2
    face_center_y = mirror_y - FACE_Y_OFFSET - FACE_HEIGHT / 2
    panel_center_y = face_center_y + FACE_HEIGHT / 2 + PANEL_Y_OFFSET

    # Draw the face (oval)
    face = Ellipse((face_center_x, face_center_y), FACE_WIDTH, FACE_HEIGHT, 
                   facecolor='moccasin', edgecolor='black', zorder=10)
    ax.add_patch(face)

    # Draw the eye (on the left side of the face)
    eye_x = face_center_x - FACE_WIDTH / 4
    eye_y = face_center_y + FACE_HEIGHT / 4
    eye = Circle((eye_x, eye_y), EYE_SIZE / 2, facecolor='white', edgecolor='black', zorder=11)
    ax.add_patch(eye)
    
    # Draw the pupil
    pupil = Circle((eye_x - EYE_SIZE / 8, eye_y), EYE_SIZE / 8, facecolor='black', zorder=12)
    ax.add_patch(pupil)

    # Draw the semi-oval panel with fixed aperture
    theta = np.linspace(0, np.pi, 100)
    x = FACE_X_POSITION + SEMI_OVAL_WIDTH / 2 * np.cos(theta)
    y = panel_center_y + FACE_HEIGHT / 4 * np.sin(theta)
    
    # Calculate the angle range for the fixed aperture
    aperture_angle = np.arcsin(APERTURE_WIDTH / SEMI_OVAL_WIDTH)
    eye_angle = np.arccos((FACE_X_POSITION - FACE_WIDTH / 4 - FACE_X_POSITION) / (SEMI_OVAL_WIDTH / 2))
    
    # Draw the left part of the semi-oval
    left_mask = theta < (eye_angle - aperture_angle / 2)
    ax.plot(x[left_mask], y[left_mask], color='gray', lw=2, zorder=9)
    
    # Draw the right part of the semi-oval
    right_mask = theta > (eye_angle + aperture_angle / 2)
    ax.plot(x[right_mask], y[right_mask], color='gray', lw=2, zorder=9)

def interactive_plot(light_height: float, light_angle: float, mirror_x_pos: float, panel_x_pos: float, face_offset: float):
    """Create an interactive plot of the mirror reflection system with a movable face."""
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.set_xlim(0, 2000)
    ax.set_ylim(0, 2000)
    ax.set_aspect('equal')
    ax.set_xlabel('X Position (mm)')
    ax.set_ylabel('Y Position (mm)')

    panel_coords = plot_panel(ax, panel_x_pos)

    mirror_end_x, mirror_end_y = calculate_mirror_end_position(mirror_x_pos, MIRROR_ANGLE_DEGREES, MIRROR_LENGTH)
    ax.plot([mirror_x_pos, mirror_end_x], [DEFAULT_MIRROR_HEIGHT, mirror_end_y], 'k-', label="Mirror")

    plot_face_and_panel(ax, DEFAULT_MIRROR_HEIGHT, face_offset)

    incident_x, incident_y = calculate_light_path(250, light_height, light_angle)

    # Check for panel intersection first
    panel_intersection = calculate_panel_intersection(incident_x, incident_y, panel_x_pos)
    if panel_intersection:
        ax.plot([incident_x[0], panel_intersection[0]], [incident_y[0], panel_intersection[1]], 'r-', label="Incident Ray (Stopped by Panel)")
    else:
        # If no panel intersection, check for mirror intersection
        reflected_coords = calculate_intersection(mirror_x_pos, incident_x, incident_y, light_angle)
        if reflected_coords:
            ax.plot([incident_x[0], reflected_coords[2][0]], [incident_y[0], reflected_coords[2][1]], 'r-', label="Incident Ray")
            
            # Check if reflected ray intersects with panel
            reflected_panel_intersection = calculate_panel_intersection(reflected_coords[0], reflected_coords[1], panel_x_pos)
            if reflected_panel_intersection:
                ax.plot([reflected_coords[2][0], reflected_panel_intersection[0]], 
                        [reflected_coords[2][1], reflected_panel_intersection[1]], 'b-', label="Reflected Ray (Stopped by Panel)")
            else:
                ax.plot(reflected_coords[0], reflected_coords[1], 'b-', label="Reflected Ray")
        else:
            ax.plot(incident_x, incident_y, 'r-', label="Incident Ray")

    plt.legend()
    plt.show()

# Interactive widgets for adjusting parameters
interactive_widget = interactive(
    interactive_plot,
    light_height=FloatSlider(value=PANEL_BASE_HEIGHT, min=PANEL_BASE_HEIGHT-FACE_HEIGHT*2.5, max=PANEL_BASE_HEIGHT, step=1, description='Light Height', layout=Layout(width='50%')),
    light_angle=FloatSlider(value=90, min=45, max=180-45, step=1, description='Light Angle', layout=Layout(width='50%')),
    mirror_x_pos=FloatSlider(value=1200, min=1100, max=1300, step=1, description='Mirror X', layout=Layout(width='50%')),
    panel_x_pos=FloatSlider(value=900, min=900, max=1000, step=1, description='Panel X', layout=Layout(width='50%')),
    face_offset=FloatSlider(value=0, min=-1, max=1, step=0.1, description='Face Offset', layout=Layout(width='50%')),
)
display(interactive_widget)

interactive(children=(FloatSlider(value=1300.0, description='Light Height', layout=Layout(width='50%'), max=13…